
# Capstone Project 

# Author : Hamidreza Salahi

# Notebook : 1


# Table of Contents
* [Introduction](#Introduction)
* [Importing Data](#Importing-Data)
* [Creating Sample Data](#Creating-Sample-Data)
* [Data Cleaning](#Data-Cleaning)
    * [Dropping NaN Columns](#Dropping-NaN-Columns)
    * [Data Dictionary](#Data-Dictionary)
    * [Dropping NaN Rows](#Dropping-NaN-Rows)
    * [Converting Non-numeric Columns to Numeric Columns](#Converting-Non-numeric-Columns-to-Numeric-Columns)
* [Feature Selection](#Feature-Selection)    
* [References](#References)

# Introduction

LendingClub is one of the largest peer-to-peer financial services company headquartered in San Francisco, California. It was the first peer-to-peer lender to register its offerings as securities with the Securities and Exchange Commission (SEC), and to offer loan trading on a secondary market. The company reported that $15.98 billion in loans had been originated through its platform up to December 31, 2015 [1]. <br>

In this project, it is assumed that I am recruited in LendingClub company as Data Scientist to answer the following business question: <br>

**Using machine learning (ML), make a loan approval predictor that identifies whether an applicant is risky or not**. <br>

Here, a risky applicant is an applicant who will not be able to pay the instalments in due time for a long period of time. In the LendingClub dataset, `Charged off` term in the load_status column refers to those applicants who has not paid their loan and has defaulted on the loan  <br>

Having a ML indicator can be very helpful for the business for two reasons: <br>

(i) Approving a risky applicant may lead to financial loss <br>
(ii) Not approving an applicant who is likely to pay their loan also leads to loss of business profit. 


In this project, the target variable is `loan_status` column. Within this column, I will concentrate only on `Fully Paid` and `Charged off` applicants i.e., `Current`, `Late (16-30 days)`, `Late (31-120 days)`, `In Grace Period`, `Issued`, `Does not meet the credit policy. Status:Fully Paid`, `Does not meet the credit policy. Status:Charged Off` and  `Default` applicants will not be considered in the analysis. 

## Importing Data

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Creating dataframe from dataset zip file
loans_df = pd.read_csv('C:\\Users\\hamid\\Desktop\\Capstone\\Data\\Loan_status_2007-2020Q3.gzip', compression='gzip', low_memory=False, index_col=0)
loans = loans_df.copy()
loans.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,1077501,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,1077430,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,1077175,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,1076863,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,1075358,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


Starting by looking at the distribution of the target variable, `loan_status` column

In [3]:
loans['loan_status'].value_counts(normalize=True)

Fully Paid                                             0.511976
Current                                                0.352425
Charged Off                                            0.123927
Late (31-120 days)                                     0.005522
In Grace Period                                        0.003428
Late (16-30 days)                                      0.000929
Issued                                                 0.000705
Does not meet the credit policy. Status:Fully Paid     0.000680
Does not meet the credit policy. Status:Charged Off    0.000260
Default                                                0.000148
Name: loan_status, dtype: float64

Only the `Fully Paid` and `Charged Off` applicants will be retained and other categories will be dropped. Except for the `Current` applicants, which contains almost 35% of the applicants, the other ignored fields combined contribut less than 1.3%  of the dataset.  <br>
One issue here is that a significant imbalance in the population of `Fully Paid` and `Charged Off` applicants is seen. This issue is addressed in the next section below. 

## Creating Sample Data

Since the original data set is too large, we create a sample data set to work with for now. The sample data set contains  10% of the original dataset which are randomly distributed. <br>
Here, I take advantage of the size of the dataset to make a balanced sample i.e., there will be about the same number of `Fully Paid` and `Charged Off` applicants. This is done by defining a new column in the dataframe, named `loans['weights']` which determines the weights of each category in the `loan_status` column. In the weights column, all the weights except for `Fully Paid` and `Charged Off` are set to 0 i.e., the sample data will not have other categories. The weights of the two remaining columns are set to 1:5 to get a balance distribution of the two categories.

In [4]:
loans['weights'] = loans['loan_status'].map({'Fully Paid': 1, 'Charged Off': 5, 'Current': 0,'Late (31-120 days)':0,\
                                              'In Grace Period': 0,'Late (16-30 days)':0,'Issued': 0,\
                                               'Does not meet the credit policy. Status:Fully Paid':0,\
                                              'Does not meet the credit policy. Status:Charged Off': 0,'Default': 0})


In [5]:
loans_sample = loans.sample(frac = 0.1, weights='weights')
loans_sample['loan_status'].value_counts(normalize=True)

Fully Paid     0.501314
Charged Off    0.498686
Name: loan_status, dtype: float64

As seen, we have a balanced dataset in the sense that there are almost equal number of `Fully Paid` and `Charged Off` applicants

In [6]:
loans_sample['loan_status'] = loans_sample.loan_status.map({'Fully Paid':0, 'Charged Off':1})

In [7]:
#Drop the id column and weight column which was introduced earlier
loans_sample.drop(columns=['weights','id'], inplace=True)

In [8]:
loans_sample.shape

(292549, 140)

In [142]:
loan_sample = loans_sample.copy()

In [143]:
loans_sample.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
12357,32025.0,32025.0,32025.0,60 months,20.55%,858.30,D,D2,office manager,7 years,MORTGAGE,100000.0,Source Verified,Sep-2019,0,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,550xx,MN,39.97,0.0,Sep-1998,700.0,704.0,1.0,73.0,NaN,14.0,0.0,29538.0,54.6%,25.0,w,0.0,0.0,35420.830000,35420.83,32025.00,3395.83,0.0,0.00,0.0000,Apr-2020,31165.89,NaN,Apr-2020,699.0,695.0,0.0,NaN,1.0,Joint App,150000.0,29.66,Source Verified,0.0,133.0,159229.0,1.0,6.0,2.0,5.0,7.0,129691.0,63.0,1.0,2.0,9348.0,58.0,54100.0,3.0,1.0,1.0,7.0,11374.0,17662.0,62.6,0.0,0.0,252.0,112.0,5.0,5.0,0.0,5.0,NaN,4.0,NaN,0.0,4.0,4.0,6.0,7.0,11.0,8.0,14.0,4.0,14.0,0.0,0.0,0.0,3.0,96.0,50.0,0.0,0.0,210073.0,159229.0,47200.0,155973.0,56342.0,685.0,689.0,Sep-2009,1.0,0.0,13.0,71.1,8.0,10.0,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
112997,11200.0,11200.0,11200.0,36 months,9.99%,361.34,B,B3,FUEL SYSTEMS MAINTENANCE,10+ years,RENT,65000.0,Verified,Oct-2015,0,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,324xx,FL,28.19,0.0,Jun-2002,685.0,689.0,0.0,26.0,NaN,8.0,0.0,9790.0,45.7%,14.0,f,0.0,0.0,13016.367888,13016.37,11200.00,1816.37,0.0,0.00,0.0000,Oct-2018,361.35,NaN,Oct-2018,684.0,680.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,725.0,72566.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21400.0,NaN,NaN,NaN,4.0,9071.0,1230.0,83.6,0.0,0.0,115.0,160.0,13.0,4.0,0.0,101.0,26.0,5.0,26.0,0.0,2.0,5.0,2.0,2.0,8.0,5.0,6.0,5.0,8.0,0.0,0.0,0.0,3.0,92.9,100.0,0.0,0.0,97239.0,72566.0,7500.0,75839.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
64094,20000.0,20000.0,20000.0,36 months,15.05%,693.80,C,C4,Registered Nurse,1 year,RENT,60000.0,V

In [144]:
loan_sample = loan_sample.reset_index(drop=True)

In [145]:
loan_sample.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,32025.0,32025.0,32025.0,60 months,20.55%,858.30,D,D2,office manager,7 years,MORTGAGE,100000.0,Source Verified,Sep-2019,0,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,550xx,MN,39.97,0.0,Sep-1998,700.0,704.0,1.0,73.0,NaN,14.0,0.0,29538.0,54.6%,25.0,w,0.0,0.0,35420.830000,35420.83,32025.00,3395.83,0.0,0.00,0.0000,Apr-2020,31165.89,NaN,Apr-2020,699.0,695.0,0.0,NaN,1.0,Joint App,150000.0,29.66,Source Verified,0.0,133.0,159229.0,1.0,6.0,2.0,5.0,7.0,129691.0,63.0,1.0,2.0,9348.0,58.0,54100.0,3.0,1.0,1.0,7.0,11374.0,17662.0,62.6,0.0,0.0,252.0,112.0,5.0,5.0,0.0,5.0,NaN,4.0,NaN,0.0,4.0,4.0,6.0,7.0,11.0,8.0,14.0,4.0,14.0,0.0,0.0,0.0,3.0,96.0,50.0,0.0,0.0,210073.0,159229.0,47200.0,155973.0,56342.0,685.0,689.0,Sep-2009,1.0,0.0,13.0,71.1,8.0,10.0,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,11200.0,11200.0,11200.0,36 months,9.99%,361.34,B,B3,FUEL SYSTEMS MAINTENANCE,10+ years,RENT,65000.0,Verified,Oct-2015,0,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,324xx,FL,28.19,0.0,Jun-2002,685.0,689.0,0.0,26.0,NaN,8.0,0.0,9790.0,45.7%,14.0,f,0.0,0.0,13016.367888,13016.37,11200.00,1816.37,0.0,0.00,0.0000,Oct-2018,361.35,NaN,Oct-2018,684.0,680.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,725.0,72566.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21400.0,NaN,NaN,NaN,4.0,9071.0,1230.0,83.6,0.0,0.0,115.0,160.0,13.0,4.0,0.0,101.0,26.0,5.0,26.0,0.0,2.0,5.0,2.0,2.0,8.0,5.0,6.0,5.0,8.0,0.0,0.0,0.0,3.0,92.9,100.0,0.0,0.0,97239.0,72566.0,7500.0,75839.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,20000.0,20000.0,20000.0,36 months,15.05%,693.80,C,C4,Registered Nurse,1 year,RENT,60000.0,Verified,Aug-2

## Data Cleaning

At a glance, it is seen that the data set has a lot of Nan values, specially in some certain columns. We will drop the columns which have more than 10% of their data missing

### Dropping NaN Columns

In [146]:
loan_sample.isna().sum()

loan_amnt                                          0
funded_amnt                                        0
funded_amnt_inv                                    0
term                                               0
int_rate                                           0
                                               ...  
hardship_loan_status                          289134
orig_projected_additional_accrued_interest    288666
hardship_payoff_balance_amount                287561
hardship_last_payment_amount                  287561
debt_settlement_flag                               0
Length: 140, dtype: int64

In [147]:
# Finding all columns with more than 10% values missing
nan_cols = [i for i in loan_sample.columns if loan_sample[i].isnull().sum() > 0.1*len(loan_sample)]

In [148]:
# Dropping nan columns 
loan_sample.drop(nan_cols , axis=1, inplace=True)
loan_sample.shape

(292549, 90)

### Data Dictionary

In [149]:
data_Dic = pd.read_excel('C:\\Users\\hamid\\Desktop\\Capstone\\Data\\LoanDataDictionary.xlsx')

In [150]:
data_Dic.head()

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...


In [151]:
# Joining the dictionary and column names
desc_loans_colms = pd.merge(pd.DataFrame({"col_name": list(loan_sample.columns)}), data_Dic, \
               how='inner', right_on='LoanStatNew', left_on='col_name')
desc_loans_colms.drop(['col_name'], axis =1, inplace=True)

In [152]:
# Displaying the dictionary defining each column
from IPython.display import display
with pd.option_context('display.max_rows', 100, 'display.max_columns', 3 , 'display.max_colwidth' , -1):
    display(desc_loans_colms)

,LoanStatNew,Description
0,loan_amnt,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
1,funded_amnt,The total amount committed to that loan at that point in time.
2,funded_amnt_inv,The total amount committed by investors for that loan at that point in time.
3,term,The number of payments on the loan. Values are in months and can be either 36 or 60.
4,int_rate,Interest Rate on the loan
5,installment,The monthly payment owed by the borrower if the loan originates.
6,grade,LC assigned loan grade
7,sub_grade,LC assigned loan subgrade
8,emp_title,The job title supplied by the Borrower when applying for the loan.*
9,emp_length,Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.


In [153]:
loan_sample.columns.shape

(90,)

* There are 90 columns in the dataset. However, looking at the data dictionary, one sees only 89 columns. The description of the column `total_rev_hi_lim` is missing in the Data Dictionary

Some of the features are related to a loan *after* the loan was approaved. As we are trying to predict a `risky` applicant based on their feature *before* they are approved, any feature related to payment of a loan and/or charging off a loan should be dropped. 

In [154]:
loan_sample.drop(columns = ['last_pymnt_d' , 'last_pymnt_amnt' , 'recoveries' , 'collection_recovery_fee' \
                            , 'debt_settlement_flag' , 'total_pymnt' , 'total_pymnt_inv' , 'total_rec_prncp'\
                           , 'total_rec_int' , 'total_rec_late_fee' , 'pymnt_plan' , 'last_credit_pull_d'] , inplace=True)

**Note**: The two columns `fico_range_high` and `fico_range_low` are very similar. In fact, the difference between the two columns in most of the cases is equal to 4 and for very rare cases it is equal to 5. 

In [155]:
(loan_sample['fico_range_high']-loan_sample['fico_range_low']).value_counts(normalize=True)

4.0    0.999884
5.0    0.000116
dtype: float64

We will take the average of the two column and replace them by their average.

In [156]:
# Replacing fico_range_low and fico_range_high by their average
loan_sample['fico_avg'] = (loan_sample['fico_range_high']+loan_sample['fico_range_low'])/2
loan_sample.drop(columns=['fico_range_high' , 'fico_range_low'] , inplace=True)

A similar situation exists for `last_fico_range_high`and `last_fico_range_low`. 

In [157]:
# Replacing last_fico_range_low and last_fico_range_high by their average
loan_sample['last_fico_avg'] = (loan_sample['last_fico_range_high']+loan_sample['last_fico_range_low'])/2
loan_sample.drop(columns=['last_fico_range_high' , 'last_fico_range_low'] , inplace=True)

### Dropping NaN Rows

In [158]:
loan_sample.dropna(axis=0, inplace=True)

### Converting Non-numeric Columns to Numeric Columns

There are some columns which are listed as object but by a careful examination, they can be converted to numeric columns. Lets take a look at all the object columns

In [159]:
pd.set_option('display.max_columns', None)
loan_sample.select_dtypes(include='object')

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,url,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,application_type,hardship_flag
0,60 months,20.55%,D,D2,office manager,7 years,MORTGAGE,Source Verified,Sep-2019,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,550xx,MN,Sep-1998,54.6%,w,Joint App,N
1,36 months,9.99%,B,B3,FUEL SYSTEMS MAINTENANCE,10+ years,RENT,Verified,Oct-2015,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,324xx,FL,Jun-2002,45.7%,f,Individual,N
2,36 months,15.05%,C,C4,Registered Nurse,1 year,RENT,Verified,Aug-2017,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,333xx,FL,May-1985,77.8%,w,Individual,N
3,36 months,11.53%,B,B5,Product Manager,< 1 year,RENT,Not Verified,Apr-2015,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,603xx,IL,Mar-2005,80.5%,w,Individual,N
4,60 months,17.27%,D,D3,SLS,10+ years,MORTGAGE,Source Verified,Jan-2016,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,933xx,CA,Sep-2004,36.2%,w,Individual,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292542,60 months,12.99%,C,C1,Resident physician,2 years,MORTGAGE,Source Verified,Sep-2014,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,292xx,SC,Jun-2007,34.7%,f,Individual,N
292543,36 months,15.57%,C,C3,Administrative Assistant,3 years,RENT,Not Verified,Apr-2019,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,900xx,CA,Apr-2010,86.2%,w,Individual,N
292544,36 months,16.14%,C,C4,Cleaner,1 year,RENT,Not Verified,Oct-2018,https://lendingclub.com/browse/loanDetail.acti...,other,Other,335xx,FL,Dec-2010,57.6%,w,Individual,N
292545,60 months,25.82%,E,E4,Teacher,7 years,RENT,Source Verified,Dec-2017,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,330xx,FL,Mar-2005,81%,w,Individual,N


Columns to be changed to numerics are: `term`, `int_rate`, `emp_length`, `revol_util`

In [160]:
# term
loan_sample.replace(to_replace=['36 months' , '60 months'], value=[36 , 60], regex=True , inplace = True)

In [161]:
# The int_rate column in not numeric. 
# Removing the % sign and converting int_rate col to a numceric col
loan_sample['int_rate']=loan_sample['int_rate'].str.replace('%', '').astype(float)

In [162]:
# The int_rate column in not numeric. 
# Removing the % sign and converting int_rate col to a numceric col
loan_sample['revol_util']=loan_sample['revol_util'].str.replace('%', '').astype(float)

In [163]:
# Emp_length
loan_sample['emp_length'].replace(to_replace=['years' , 'year' , '<' , '\+'], value='', regex=True, inplace = True)
loan_sample['emp_length']=loan_sample['emp_length'].astype(int)

In the numeric emp_length column, the 10+ years is changed to 10 and <1 year is changed to 1 but I will keep in mind that 10 represents 10+ and 1 represents 1 year or less of emp_length.

In [164]:
loan_sample['issue_year'] = pd.to_datetime(loan_sample['issue_d']).dt.year
loan_sample.drop(columns = 'issue_d' , inplace=True)

In [165]:
loan_sample['earliest_cr_line_year'] = pd.to_datetime(loan_sample['earliest_cr_line']).dt.year
loan_sample.drop(columns = 'earliest_cr_line' , inplace=True)

In [166]:
loan_sample['initial_list_status'].value_counts()

w    156481
f     78546
Name: initial_list_status, dtype: int64

In [167]:
loan_sample['initial_list_status'] = loan_sample['initial_list_status'].map({'w': 1, 'f': 0})

In [168]:
loan_sample['application_type'].value_counts()

Individual    225949
Joint App       9078
Name: application_type, dtype: int64

In [169]:
loan_sample['application_type'] = loan_sample['application_type'].map({'Joint App': 1, 'Individual': 0})

In [170]:
loan_sample['hardship_flag'].value_counts()

N    234870
Y       157
Name: hardship_flag, dtype: int64

In [171]:
loan_sample['hardship_flag'] = loan_sample['hardship_flag'].map({'Y': 1, 'N': 0})

The `url` and `zid_code` are also not usefull features because they do not provide us with any information. We just drop them.

In [172]:
loan_sample.drop(columns=['url' , 'zip_code'] , inplace = True)

In [173]:
loan_sample.shape[1]

74

In [174]:
loan_sample.select_dtypes(include='object')

,grade,sub_grade,emp_title,home_ownership,verification_status,purpose,title,addr_state
0,D,D2,office manager,MORTGAGE,Source Verified,debt_consolidation,Debt consolidation,MN
1,B,B3,FUEL SYSTEMS MAINTENANCE,RENT,Verified,debt_consolidation,Debt consolidation,FL
2,C,C4,Registered Nurse,RENT,Verified,debt_consolidation,Debt consolidation,FL
3,B,B5,Product Manager,RENT,Not Verified,credit_card,Credit card refinancing,IL
4,D,D3,SLS,MORTGAGE,Source Verified,debt_consolidation,Debt consolidation,CA
...,...,...,...,...,...,...,...,...
292542,C,C1,Resident physician,MORTGAGE,Source Verified,credit_card,Credit card refinancing,SC
292543,C,C3,Administrative Assistant,RENT,Not Verified,debt_consolidation,Debt consolidation,CA
292544,C,C4,Cleaner,RENT,Not Verified,other,Other,FL
292545,E,E4,Teacher,RENT,Source Verified,credit_card,Credit card refinancing,FL


### Feature Selection

The importance of the numerical features is determined based on their correlation with the target column i.e., loan_status column.

In [175]:
corr_features = np.abs(loan_sample.corr()['loan_status']).sort_values(ascending = False)
corr_features

loan_status         1.000000
last_fico_avg       0.657561
int_rate            0.313743
term                0.210256
fico_avg            0.163653
                      ...   
acc_now_delinq      0.000561
num_tl_120dpd_2m    0.000415
out_prncp                NaN
out_prncp_inv            NaN
policy_code              NaN
Name: loan_status, Length: 66, dtype: float64

We can introduce a threshold for minimum of a correction a variable should have in order to keep it for modeling. Here, I choose the threshold to be 0.02 i.e., any feature that has a correlation less than 0.02 will be dropped.

In [176]:
important_numeric_features = corr_features[corr_features>0.02].index
list(important_numeric_features)

['loan_status',
 'last_fico_avg',
 'int_rate',
 'term',
 'fico_avg',
 'acc_open_past_24mths',
 'bc_open_to_buy',
 'num_tl_op_past_12m',
 'funded_amnt',
 'loan_amnt',
 'funded_amnt_inv',
 'tot_hi_cred_lim',
 'dti',
 'avg_cur_bal',
 'total_bc_limit',
 'mort_acc',
 'tot_cur_bal',
 'mo_sin_old_rev_tl_op',
 'installment',
 'total_rev_hi_lim',
 'inq_last_6mths',
 'num_rev_tl_bal_gt_0',
 'num_actv_rev_tl',
 'mo_sin_rcnt_tl',
 'bc_util',
 'mo_sin_rcnt_rev_tl_op',
 'mths_since_recent_bc',
 'percent_bc_gt_75',
 'earliest_cr_line_year',
 'revol_util',
 'num_actv_bc_tl',
 'annual_inc',
 'num_op_rev_tl',
 'mo_sin_old_il_acct',
 'pub_rec_bankruptcies',
 'revol_bal',
 'num_sats',
 'open_acc',
 'issue_year',
 'pub_rec',
 'application_type',
 'emp_length']

The difference between fico_range_low and fico_range_high is 4 for 99% of the sample data (shown below). Later on in this notebook, a new column called fico_avg will be created as fico_avg=(fico_range_low+fico_range_high)/2

In [177]:
important_category_features = list(loan_sample.select_dtypes(['object']).columns)
important_category_features

['grade',
 'sub_grade',
 'emp_title',
 'home_ownership',
 'verification_status',
 'purpose',
 'title',
 'addr_state']

We can now combine the important numeric and categorical features to get a list of important features.

In [178]:
important_features = [*important_numeric_features , *important_category_features]
important_features

['loan_status',
 'last_fico_avg',
 'int_rate',
 'term',
 'fico_avg',
 'acc_open_past_24mths',
 'bc_open_to_buy',
 'num_tl_op_past_12m',
 'funded_amnt',
 'loan_amnt',
 'funded_amnt_inv',
 'tot_hi_cred_lim',
 'dti',
 'avg_cur_bal',
 'total_bc_limit',
 'mort_acc',
 'tot_cur_bal',
 'mo_sin_old_rev_tl_op',
 'installment',
 'total_rev_hi_lim',
 'inq_last_6mths',
 'num_rev_tl_bal_gt_0',
 'num_actv_rev_tl',
 'mo_sin_rcnt_tl',
 'bc_util',
 'mo_sin_rcnt_rev_tl_op',
 'mths_since_recent_bc',
 'percent_bc_gt_75',
 'earliest_cr_line_year',
 'revol_util',
 'num_actv_bc_tl',
 'annual_inc',
 'num_op_rev_tl',
 'mo_sin_old_il_acct',
 'pub_rec_bankruptcies',
 'revol_bal',
 'num_sats',
 'open_acc',
 'issue_year',
 'pub_rec',
 'application_type',
 'emp_length',
 'grade',
 'sub_grade',
 'emp_title',
 'home_ownership',
 'verification_status',
 'purpose',
 'title',
 'addr_state']

In [179]:
loan_sample = loan_sample[important_features]

In [180]:
#Saving the sample data
loan_sample.to_csv('C:\\Users\\hamid\\Desktop\\Capstone\\Data\\loan_sample.csv' , index=False)

# References
[1] https://en.wikipedia.org/wiki/LendingClub